In [55]:
import requests as r
import pandas as pd
import getpass
import json
from bs4 import BeautifulSoup as bs
import re
import lxml

In [13]:
api_key = 'R9uFqEv'

In [14]:
url = 'https://strainapi.evanbusse.com/' + api_key + '/searchdata/effects'

request = r.get(url)
json_request = request.json()

effects = pd.DataFrame(json_request)
effects.head()

,effect,type
0,Relaxed,positive
1,Dizzy,negative
2,Hungry,positive
3,Euphoric,positive
4,Happy,positive


In [16]:
url = 'https://strainapi.evanbusse.com/' + api_key + '/searchdata/flavors'

request = r.get(url)
json_request = request.json()

flavors = pd.DataFrame(json_request)
flavors.head()

,0
0,Earthy
1,Chemical
2,Pine
3,Spicy/Herbal
4,Pungent


In [21]:
url = 'https://strainapi.evanbusse.com/' + api_key + '/strains/search/all'

request = r.get(url)
json_request = request.json()

strains = pd.DataFrame(json_request).T.reset_index().rename(columns = {'index': 'strain'})
strains.head()

,strain,id,race,flavors,effects
0,Afpak,1,hybrid,"[Earthy, Chemical, Pine]","{'positive': ['Relaxed', 'Hungry', 'Happy', 'S..."
1,African,2,sativa,"[Spicy/Herbal, Pungent, Earthy]","{'positive': ['Euphoric', 'Happy', 'Creative',..."
2,Afternoon Delight,3,hybrid,"[Pepper, Flowery, Pine]","{'positive': ['Relaxed', 'Hungry', 'Euphoric',..."
3,Afwreck,4,hybrid,"[Pine, Earthy, Flowery]","{'positive': ['Relaxed', 'Happy', 'Creative', ..."
4,Agent Orange,5,hybrid,"[Citrus, Orange, Sweet]","{'positive': ['Relaxed', 'Euphoric', 'Happy', ..."


In [22]:
strains.shape[0]

1970

In [46]:
urls = list(map(lambda x: 'https://strainapi.evanbusse.com/' + api_key + '/strains/data/desc/' + str(x), range(1, strains.shape[0] + 1)))
json_requests = list(map(lambda x: r.get(x).json()['desc'], urls))

descriptions = pd.Series(json_requests)
descriptions.head()

0    {'desc': 'Afpak, named for its direct Afghani ...
1    {'desc': 'African refers to the indigenous var...
2    {'desc': 'Afternoon Delight, created by Colora...
3    {'desc': 'Afwreck is a hybrid cross of Afghani...
4    {'desc': 'Don’t let the name scare you! The on...
dtype: object

In [52]:
strains_full = pd.concat([strains, descriptions], axis = 1)
strains_full

,strain,id,race,flavors,effects,0
0,Afpak,1,hybrid,"[Earthy, Chemical, Pine]","{'positive': ['Relaxed', 'Hungry', 'Happy', 'S...","Afpak, named for its direct Afghani and Pakist..."
1,African,2,sativa,"[Spicy/Herbal, Pungent, Earthy]","{'positive': ['Euphoric', 'Happy', 'Creative',...",African refers to the indigenous varieties of ...
2,Afternoon Delight,3,hybrid,"[Pepper, Flowery, Pine]","{'positive': ['Relaxed', 'Hungry', 'Euphoric',...","Afternoon Delight, created by Colorado Seed In..."
3,Afwreck,4,hybrid,"[Pine, Earthy, Flowery]","{'positive': ['Relaxed', 'Happy', 'Creative', ...",Afwreck is a hybrid cross of Afghani and Train...
4,Agent Orange,5,hybrid,"[Citrus, Orange, Sweet]","{'positive': ['Relaxed', 'Euphoric', 'Happy', ...",Don’t let the name scare you! The only herbici...
...,...,...,...,...,...,...
1965,Zeus OG,2158,hybrid,"[Earthy, Woody, Pine]","{'positive': ['Relaxed', 'Happy', 'Energetic',...","The Flav combines the oily, thick rich fruity ..."
1966,Zkittlez,2159,indica,"[Sweet, Berry, Grape]","{'positive': ['Relaxed', 'Euphoric', 'Happy', ...",Colorado Seed Inc.’s “The Fork” is dubbed as s...
1967,Zombie OG,2160,indica,"[Earthy, Pungent, Sweet]","{'positive': ['Relaxed', 'Euphoric', 'Happy', ...",The Grunk is a perfect representation of its p...
1968,Zoom Pie,2161,indica,"[Berry, Earthy, Pungent]","{'positive': ['Relaxed', 'Hungry', 'Happy', 'U...","Originally bred by T.H. Seeds in Amsterdam, Th..."


In [53]:
strains_full.to_csv('green-api.csv', index = False)

In [169]:
popular_soup = bs(r.get('https://www.leafly.com/explore').content, 'lxml')

links = pd.Series(map(lambda x: 'https://www.leafly.com' + str(x).split('"')[3], popular_soup.find_all('a', 'ga_Explore_Strain_Tile')))[:20]
soups = pd.Series(map(lambda x: bs(r.get(x).content, 'lxml'), links))

ratings = pd.Series(map(lambda x: str(x.find_all('span', 'rating__count')).split('>')[1].split()[0], soups))
rating_sizes = pd.Series(map(lambda x: str(x.find_all('span', 'rating__count')).split('>')[1].split()[2], soups))
types = pd.Series(map(lambda x: str(x.find_all('div', 'strain-tile-header')).split('>')[1].split('<')[0], soups))
aliases = pd.Series(map(lambda x: str(x.find_all('div', 'strain-tile-body copy-boldest')).split('>')[1].split('<')[0], soups))
names = pd.Series(map(lambda x: str(x.find_all('div', 'strain-tile-footer')).split('>')[1].split('<')[0], soups))
positive_effects = pd.Series(map(lambda x: str(x.find_all('div', 'histogram-label')).replace('<', '>').split('>')[2::4][:5], soups))
medicals_effects = pd.Series(map(lambda x: str(x.find_all('div', 'histogram-label')).replace('<', '>').split('>')[2::4][6:10], soups))
negatives_effects = pd.Series(map(lambda x: str(x.find_all('div', 'histogram-label')).replace('<', '>').split('>')[2::4][11:], soups))

strains_popular = pd.concat([names, aliases, types, ratings, rating_sizes, positive_effects, negatives_effects, medicals_effects], axis = 1)
strains_popular.columns = ['name', 'alias', 'type', 'rating', 'rating_size', 'positive_effects', 'negatives_effects', 'medicals_effects']
strains_popular

,name,alias,type,rating,rating_size,positive_effects,negatives_effects,medicals_effects
0,Blue Dream,Bd,Hybrid,4.4,12894,"[Happy, Relaxed, Euphoric, Uplifted, Creative]","[Dry Eyes, Anxious, Dizzy, Paranoid]","[Depression, Pain, Fatigue, Headaches]"
1,Sour Diesel,Sd,Sativa,4.4,7434,"[Happy, Energetic, Uplifted, Euphoric, Creative]","[Dry Eyes, Anxious, Paranoid, Dizzy]","[Depression, Pain, Fatigue, Headaches]"
2,GSC,Gsc,Hybrid,4.5,6847,"[Happy, Relaxed, Euphoric, Uplifted, Creative]","[Dry Eyes, Anxious, Dizzy, Paranoid]","[Depression, Pain, Insomnia, Lack of Appetite]"
3,Green Crack,Gc,Sativa,4.4,6145,"[Energetic, Happy, Uplifted, Euphoric, Focused]","[Dry Eyes, Anxious, Paranoid, Dizzy]","[Depression, Fatigue, Pain, Headaches]"
4,OG Kush,Ogk,Hybrid,4.4,5015,"[Relaxed, Happy, Euphoric, Uplifted, Sleepy]","[Dry Eyes, Dizzy, Anxious, Paranoid]","[Depression, Pain, Insomnia, Lack of Appetite]"
5,Granddaddy Purple,Gdp,Indica,4.5,4618,"[Relaxed, Sleepy, Euphoric, Happy, Hungry]","[Dry Eyes, Dizzy, Anxious, Paranoid]","[Pain, Insomnia, Depression, Lack of Appetite]"
6,Jack Herer,Jh,Sativa,4.4,4152,"[Happy, Energetic, Uplifted, Creative, Euphoric]","[Dry Eyes, Anxious, Paranoid, Dizzy]","[Depression, Pain, Fatigue, Headaches]"
7,Original Glue,Gg4,Hybrid,4.7,3936,"[Relaxed, Euphoric, Happy, Uplifted, Sleepy]","[Dry Eyes, Paranoid, Anxious, Dizzy]","[Pain, Depression, Insomnia, Lack of Appetite]"
8,Blue Dream,Bd,Hybrid,4.4,12894,"[Happy, Relaxed, Euphoric, Uplifted, Creative]","[Dry Eyes, Anxious, Dizzy, Paranoid]","[Depression, Pain, Fatigue, Headaches]"
9,Sour Diesel,Sd,Sativa,4.4,7434,"[Happy, Energetic, Uplifted, Euphoric, Creative]","[Dry Eyes, Anxious, Paranoid, Dizzy]","[Depression, Pain, Fatigue, Headaches]"


In [170]:
strains_popular.to_csv('green-scrape.csv', index = False)